In [1]:
import sympy as sp

# Define the symbols
alpha, beta, gama, X, Y, Z, S_x, S_y, S_z = sp.symbols(
    'alpha beta gama X Y Z S_x S_y S_z', real=True)

ff = 0.008
# Sip and TCP
Sip = sp.Matrix([S_x, S_y, S_z])
TCP = sp.Matrix([X, Y, Z])

# Rotation matrices Rx, Ry, Rz
Rx = sp.Matrix([[1, 0, 0], 
                [0, sp.cos(alpha), -sp.sin(alpha)], 
                [0, sp.sin(alpha), sp.cos(alpha)]])
Ry = sp.Matrix([[sp.cos(beta), 0, sp.sin(beta)], 
                [0, 1, 0], 
                [-sp.sin(beta), 0, sp.cos(beta)]])
Rz = sp.Matrix([[sp.cos(gama), -sp.sin(gama), 0], 
                [sp.sin(gama), sp.cos(gama), 0], 
                [0, 0, 1]])
R = sp.simplify(Rx * Ry * Rz)
L = 309.5 * 1e-3

# Vector v and calculations for x and o
v = sp.Matrix([0, 0, -1])
n = sp.Matrix([0, 1, 0])
x = sp.Matrix([X, Y, Z]) - L * R * v
o = R * v

# Calculate 'a' using dot products
a = sp.sqrt(((-R.row(0).dot(v) * Z) / (R.row(2).dot(v)))**2 + (-(R.row(1).dot(v) * Z) / (R.row(2).dot(v)))**2 + Z * Z)

# Calculate Q
Q = a * o + TCP

# Project Sip onto the line defined by o and Q
Sip_proj = Sip + (o.dot(Q) - o.dot(Sip)) * (x - Sip) / (o.dot(x) - o.dot(Sip))

# s_2 vector and s_1 as the cross product of o and s_2
s_2 = R * n
s_1 = o.cross(s_2)

# Calculate k_1 and k_2
k_1 = (Sip_proj - Q).dot(s_1) / s_1.norm()**2
k_2 = (Sip_proj - Q).dot(s_2) / s_2.norm()**2


# Calculate image plane coordinates f_1 and f_2
f_1 = k_1 * ff / (a + L - ff)
f_2 = k_2 * ff / (a + L - ff)

In [2]:

M = sp.Matrix(5, 3, (0, 0, 0, 0.05, 0.05, 0, 0.05, -0.05, 0, -0.05, -0.05, 0, -0.05, 0.05, 0))
# Define the matrix to store the results
results = sp.Matrix(10, 1, [0] * 10)

# Define the loop
for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})

    # Store the results in the matrix
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3

In [3]:
import inspect
v = sp.Matrix([X, Y, Z, alpha, beta, gama])
results_lamb = sp.lambdify(v, results, modules='numpy', cse=True)

print(inspect.getsource(results_lamb))

def _lambdifygenerated(X, Y, Z, alpha, beta, gama):
    x0 = cos(beta)
    x1 = x0**2
    x2 = cos(alpha)
    x3 = sin(gama)
    x4 = x2*x3
    x5 = sin(beta)
    x6 = sin(alpha)
    x7 = cos(gama)
    x8 = x4*x5 + x6*x7
    x9 = x1*x4 + x5*x8
    x10 = x0*x6
    x11 = 0.3095*x10
    x12 = Y - x11
    x13 = X + 0.3095*x5
    x14 = x13*x5
    x15 = x0*x2
    x16 = Z + 0.3095*x15
    x17 = x15*x16
    x18 = -x0*x12*x6 + x14 + x17
    x19 = -1/x18
    x20 = Z**2
    x21 = x20/x2**2
    x22 = sqrt(x20 + x21*x6**2 + x21*x5**2/x1)
    x23 = x22*x5
    x24 = X - x23
    x25 = x24*x5
    x26 = Y + x10*x22
    x27 = x15*x22
    x28 = x15*(Z - x27)
    x29 = -x0*x26*x6 + x25 + x28
    x30 = -x29
    x31 = x12*x19*x30 - x26
    x32 = -x2*x7 + x3*x5*x6
    x33 = -x32
    x34 = x1*x3*x6 - x33*x5
    x35 = x19*x30
    x36 = -Z + x27
    x37 = x16*x35 + x36
    x38 = x10*x8 + x15*x33
    x39 = -X + x23
    x40 = x13*x35 + x39
    x41 = 8.0/(x22 + 0.3015)
    x42 = x41/(x34**2 + x38**2 + x9**2)
    x4

In [33]:

f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Define the symbols
f_obs = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))
diff = results - f_obs
F_obs = (diff[0] * diff[0] + diff[1] * diff[1] +
         diff[2] * diff[2] + diff[3] * diff[3] +
         diff[4] * diff[4] + diff[5] * diff[5] +
         diff[6] * diff[6] + diff[7] * diff[7] +
         diff[8] * diff[8] + diff[9] * diff[9])


In [35]:
import inspect
v = sp.Matrix([X, Y, Z, alpha, beta, gama, f_obs1, f_obs2, f_obs3,
              f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10])

args = sp.Matrix([X, Y, Z, alpha, beta, gama])
hess_Fobs = sp.hessian(F_obs, args)
F_obs_lambd = sp.lambdify(v, hess_Fobs, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

def _lambdifygenerated(X, Y, Z, alpha, beta, gama, f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10):
    x0 = sin(alpha)
    x1 = cos(gama)
    x2 = x0*x1
    x3 = sin(beta)
    x4 = sin(gama)
    x5 = cos(alpha)
    x6 = x4*x5
    x7 = x3*x6
    x8 = x2 + x7
    x9 = cos(beta)
    x10 = x5*x9
    x11 = 0.3095*x10
    x12 = Z + x11
    x13 = 0.3095*x3
    x14 = X + x13
    x15 = x14*x3
    x16 = x10*x12
    x17 = x0*x9
    x18 = 0.3095*x17
    x19 = Y - x18
    x20 = -x0*x19*x9 + x15 + x16
    x21 = -x20
    x22 = x21**(-1.0)
    x23 = x22*x3
    x24 = x12*x23
    x25 = Z**2
    x26 = x0**2
    x27 = x5**2
    x28 = x27**(-1.0)
    x29 = x26*x28
    x30 = x25*x29
    x31 = x25*x28
    x32 = x3**2
    x33 = x9**2
    x34 = x32/x33
    x35 = x31*x34
    x36 = x25 + x35
    x37 = x30 + x36
    x38 = sqrt(x37)
    x39 = x3*x38
    x40 = -x39
    x41 = X + x40
    x42 = x3*x41
    x43 = x38*x9
    x44 = x43*x5
    x45 = -x44
    x46 = Z + x45
    x47 = x10*x4

In [37]:
import numba
F_obs_lambd_numba = numba.jit(F_obs_lambd, nopython=True, fastmath=True)

In [41]:
%%timeit
F_obs_lambd_numba(0.0, 0.0,1e-16,0.0,0.0,0.0, 0,0,0,0,0,0,0,0,0,0)

1.67 µs ± 12.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [3]:
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

tmp = []
for i in range(10):
    tmp.append(sp.hessian(results[i], v))



In [4]:
tmp1 = sp.Array(tmp, (6, 6, 10))

In [6]:
import inspect
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

F_obs_lambd = sp.lambdify(v, tmp1, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

def _lambdifygenerated(X, Y, Z, alpha, beta, gama):
    x0 = cos(beta)
    x1 = x0**2
    x2 = sin(gama)
    x3 = cos(alpha)
    x4 = x2*x3
    x5 = sin(beta)
    x6 = sin(alpha)
    x7 = cos(gama)
    x8 = x6*x7
    x9 = x4*x5
    x10 = x8 + x9
    x11 = x10*x5
    x12 = x1*x4 + x11
    x13 = x0*x6
    x14 = 0.3095*x13
    x15 = Y - x14
    x16 = 0.3095*x5
    x17 = X + x16
    x18 = x17*x5
    x19 = x0*x3
    x20 = 0.3095*x19
    x21 = Z + x20
    x22 = x19*x21
    x23 = -x0*x15*x6 + x18 + x22
    x24 = -x23
    x25 = x24**(-2.0)
    x26 = x5**2
    x27 = 2*x26
    x28 = x25*x27
    x29 = Z**2
    x30 = x6**2
    x31 = x3**2
    x32 = x31**(-1.0)
    x33 = x30*x32
    x34 = x29*x33
    x35 = x29*x32
    x36 = x26/x1
    x37 = x35*x36
    x38 = x29 + x37
    x39 = x34 + x38
    x40 = sqrt(x39)
    x41 = x40*x5
    x42 = -x41
    x43 = X + x42
    x44 = x43*x5
    x45 = x0*x40
    x46 = x3*x45
    x47 = -x46
    x48 = Z + x47
    x49 = x19*x48
    x50 = x45*x6
    x51 = Y + x50
    x52

In [24]:
from tmp_hesf import hess

In [30]:
%%timeit
hess(0.0, 0.0,1e-16,0.0,0.0,0.0)

2.99 µs ± 40 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [23]:
import numpy as np

res = np.array(F_obs_lambd(0.0,0.0,1e-16,0.0,0.0,0.0))

np.round(res[:,:,9],1)

array([[  0. ,   0. ,   0. ,   0. ,  88. ,   0. ],
       [  0. ,   0. ,  -4.4,   1.3,   0. ,   0. ],
       [ -1.3,   0. ,   0. ,  -1.4,   0. ,  -1.3],
       [ -8.6,  -8.6,  -0.2,  -4.3,   0. ,  -0.2],
       [ -0.2,   0. ,  -4.4,   0.1,   0. , -26.5],
       [ -0.1, -26.5,  26.5, -25.1,   4.3,  -1.3]])

In [ ]:
import inspect
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

gradient_F_obs = sp.derive_by_array(results, v)
F_obs_lambd = sp.lambdify(v, results, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

In [ ]:
gradient_F_obs.shape

In [ ]:
import inspect
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_F_obs = sp.derive_by_array(f_1, v)

hessian_F_obs = sp.hessian(f_1, v)
args = [alpha, beta, gama, X, Y, Z]
F_obs_lambd = sp.lambdify(args, hessian_F_obs, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

In [ ]:
import sympy as sp

# Define the symbols
S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15 = sp.symbols("S1:16", real=True)

# Create the matrix
M = sp.Matrix(5, 3, (S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15))

# Print the matrix
print(M)

In [ ]:
# Define the matrix to store the results
results = sp.Matrix(10, 1, [0] * 10)

# Define the loop
for i in range(5):
    # Substitute the symbols from M for S_x, S_y, S_z in f_1 and f_2
    f1_i = f_1.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})
    f2_i = f_2.subs({S_x: M[i, 0], S_y: M[i, 1], S_z: M[i, 2]})

    # Store the results in the matrix
    results[2 * i, 0] = f1_i * 1e3
    results[2 * i + 1, 0] = f2_i * 1e3

In [ ]:
import sympy as sp

f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10 = sp.symbols('f_obs1:11', real=True)

# Define the symbols
f_obs = sp.Matrix(10, 1, (f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10))



In [ ]:
diff = results - f_obs
F_obs = (diff[0] * diff[0] + diff[1] * diff[1] +
         diff[2] * diff[2] + diff[3] * diff[3] +
         diff[4] * diff[4] + diff[5] * diff[5] +
         diff[6] * diff[6] + diff[7] * diff[7] +
         diff[8] * diff[8] + diff[9] * diff[9])
F_obs

In [ ]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff, 
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
F_obs_lambd = sp.lambdify(args, F_obs, modules='numpy', cse=True)

print(inspect.getsource(F_obs_lambd))

In [ ]:
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v
gradient_F_obs = sp.derive_by_array(F_obs, v)

In [ ]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
gradient_F_obs_lambd = sp.lambdify(args, gradient_F_obs, modules='numpy', cse=True)

print(inspect.getsource(gradient_F_obs_lambd))

In [ ]:
import sympy as sp
# Define the vector of symbols
v = sp.Matrix([X, Y, Z, alpha, beta, gama])

# Compute the gradient of f_1 with respect to the vector v

# Create a 6x6 matrix in sympy
hessian_F_obs = sp.Matrix(6, 6, [0] * 36)

for i in range(6):
    for j in range(6):
        print(f"Done with {i}, {j}")
        hessian_F_obs[i, j] = sp.diff(gradient_F_obs[i], v[j])

In [ ]:
print(hessian_F_obs[0,0])

In [ ]:
import inspect

args = [alpha, beta, gama, X, Y, Z, L, v1, v2, v3, n1, n2, n3, ff,
        S1, S2, S3, S4, S5, S6, S7, S8, S9, S10, S11, S12, S13, S14, S15,
        f_obs1, f_obs2, f_obs3, f_obs4, f_obs5, f_obs6, f_obs7, f_obs8, f_obs9, f_obs10]
hessian_F_obs_lambd = sp.lambdify(args, hessian_F_obs, modules='numpy', cse=True)

print(inspect.getsource(hessian_F_obs_lambd))

In [ ]:
import numpy as np

res_np = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        xpr_term, xpr_res = sp.cse(hessian_F_obs[i, j])
        res_np[i, j] = len(xpr_term)

res_np

In [ ]:
list(range(6,6))

In [ ]:
for i in range(6):
    for j in range(i+1,6):
        hessian_F_obs[i, j] = 0

In [ ]:
xpr_term, xpr_res = sp.cse(hessian_F_obs)

In [ ]:
len(xpr_term)

In [ ]:
subs_dict = {}
for var, subexpr in xpr_term:
    subs_dict[var] = subexpr
subs_dict

In [ ]:
subs_dict1 = {}
for var, subexpr in xpr_term:
    if str(subexpr).find('Derivative') != -1:
        subs_dict1[var] = subexpr.subs(subs_dict)
subs_dict1

In [ ]:
zero_dict = {key: 0 for key in subs_dict1.keys()}
zero_dict

In [ ]:
# Drop subs_dict entries from subs_dict1.keys()
for key in zero_dict:
    if key in subs_dict:
        subs_dict.pop(key)

# Substitute each entry in subs_dict for 0 in subs_dict1
for key in subs_dict:
    subs_dict[key] = subs_dict[key].subs(zero_dict)


In [ ]:
for key in subs_dict:
    if subs_dict[key] == 0:
        print(key)
        zero_dict[key] = 0


In [ ]:
subs_dict2 = {}
for var, subexpr in xpr_term:
    if str(subexpr).find('**') != -1:
        if str(subexpr.subs(subs_dict))[0:4] == 'sign':
            subs_dict2[var] = subexpr.subs(subs_dict)
subs_dict2

In [ ]:
one_dict = {key: 1 for key in subs_dict2.keys()}
one_dict

In [ ]:
# Drop subs_dict entries from subs_dict1.keys()
for key in one_dict:
    if key in subs_dict:
        subs_dict.pop(key)

# Substitute each entry in subs_dict for 0 in subs_dict1
for key in subs_dict:
    subs_dict[key] = subs_dict[key].subs(one_dict)

In [ ]:
for key in subs_dict:
    if subs_dict[key] == 1:
        print(key)
        one_dict[key] = 1

In [ ]:
with open('tmp_new2.txt', 'w') as f:
    for key in subs_dict:
        f.write(str(key) + " = " + str(subs_dict[key]) + ";\n")

In [ ]:
xpr_res[0][1,1][0]

In [ ]:

subs_res = list()
for i in range(6):
    tmp = list()
    for j in range(6):
        try:
            tmp.append(xpr_res[0][i,j][0].subs(zero_dict).subs(one_dict))
        except:
            tmp.append(0)
    subs_res.append(tmp)


In [ ]:
with open('tmp_new2.txt', 'a') as f:
    for i in range(6):
        for j in range(6):
            f.write("res(" + str(i+1) + "," + str(j+1) + ") = " + str(subs_res[i][j]) + ";\n")

In [ ]:
from sympy import symbols, Function, diff

# Define the variable
x = symbols('x')

# Define the function f as an unspecified function of x
f = Function('f')(x)

# Take the derivative of f with respect to x
dfdx = diff(f, x)

# Display the derivative
dfdx

In [ ]:
from sympy import symbols, Function, Matrix, diff, Sum

# Define the variables
x1, x2, x3, x4, x5, x6 = symbols('x1 x2 x3 x4 x5 x6')

# Define a generic function f that will represent our vector function components
F = Function('f')

# Create a vector of these placeholder functions, each dependent on all variables
f_vector = Matrix([Function(f"f_{i+1}")(x1, x2, x3, x4, x5, x6) for i in range(10)])

# Assume we have another vector g for the purpose of creating a sum of squared differences
g_vector = Matrix(10, 1, lambda i, _: symbols(f'g{i+1}'))
f_vector

In [ ]:
sum_squared_diffs = sum((f_vector - g_vector).applyfunc(lambda x: x**2))
sum_squared_diffs

In [ ]:

# To illustrate the gradient computation, we'd normally take the derivative
# of sum_squared_diffs with respect to each variable x1, x2, ..., x6
# This will be symbolic since f is unspecified
gradient = Matrix([diff(sum_squared_diffs, var) for var in (x1, x2, x3, x4, x5, x6)])

gradient